In [83]:
from collections import defaultdict
from matplotlib import pyplot as plt
import numpy as np
import operator
import pickle

import src.fusiontools as fusiontools
import src.scoring as scoring

%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [93]:
# Load best recency model
recency_rank_dic_path = 'ranks_test/recency_beta_7000_rec-all'
with open(recency_rank_dic_path, 'rb') as infile:
    recency_ranks = pickle.load(infile)

    
# # Load best knn model
# knn_rank_dic_path = 'ranks_test/recency_beta_7000_rec-all'
# with open(knn_rank_dic_path, 'rb') as infile:
#     knn_ranks = pickle.load(infile)


In [92]:
for values in recency_ranks.values():
    print(len(values))

99
152
57
412
1
272
14
14
102
200
61
1242
412
243
57
202
242
200
183
200
147
147
133
81
140
147
272
412
152
76
284
120
81
412
440
184
1242
98
184
184
674
140
102
351
351
102
97
224
130
210
14
133
412
202
1
394
360
123
224
243
94
203
184
412
351
120
394
394
412
134
224
200
102
202
167
540
133
183
272
133
159
159
130
76
136
360
147
167
352
440
456
242
381
221
456
133
221
284
440
143
456
1242
1242
272
202
72
456
67
65
1242
272
50
50
283
50
50
412
351
140
177
1242
1242
184
284
371
1242
391
352
83
272
65
202
83
284
247
184
247
203
371
371
371
371
76
174
174
283
120
540
143
1
174
202
184
540
540
86
120
120
349
116
351
352
352
188
76
116
116
143
116
183
116
116
116
167
202
202
456
116
202
116
116
143
243
394
188
202
202
202
243
36
83
243
1
40
352
36
352
130
839
352
147
97
412
183
405
224
183
140
224
391
202
247
202
184
221
203
352
405
183
143
76
76
97
134
381
88
272
133
405
133
352
175
36
36
36
188
133
36
98
133
133
140
99
139
133
133
133
133
133
99
272
98
143
133
133
283
540
139
67
412
394
1

In [82]:
models = [recency_ranks, knn_ranks]

fusion_dic = fusiontools.reciprocal_rerank(models, 10)
print(fusion_dic)

            
                

{172034: ['james.d.steffes@enron.com', 'steven.j.kean@enron.com', 'louise.kitchen@enron.com', 'richard.shapiro@enron.com', 'mark.koenig@enron.com', 'john.lavorato@enron.com', 'kenneth.lay@enron.com', 'james.derrick@enron.com', 'sarah.novosel@enron.com', 'greg.whalley@enron.com', 'harry.kingerski@enron.com', 'rick.buy@enron.com', 'mark.palmer@enron.com', 'karen.denne@enron.com', 'hunter.s.shively@enron.com', 'rogers.herndon@enron.com', 'kay.chapman@enron.com', 'e..haedicke@enron.com', 'david.w.delainey@enron.com', 'kimberly.hillis@enron.com', 'tammie.schoppe@enron.com', 'liz.taylor@enron.com', 'leslie.lawner@enron.com', 'janine.migden@enron.com', 'susan.lindberg@enron.com', 'david.forster@enron.com', 'paula.rieker@enron.com', 'jeff.dasovich@enron.com', 'donna.fulton@enron.com', 'richard.b.sanders@enron.com'], 85335: ['marie.heard@enron.com', 'sara.shackleton@enron.com', 'tracy.ngo@enron.com', 'elizabeth.sager@enron.com', 's..bradford@enron.com', 'barry.tycholiz@enron.com', 'l..nicolay@e

{40962: ['terry.kowalke@enron.com',
  'steven.j.kean@enron.com',
  'john.buchanan@enron.com',
  'ginger.dernehl@enron.com',
  'james.d.steffes@enron.com',
  'richard.shapiro@enron.com',
  'sarah.novosel@enron.com',
  'sandra.mccubbin@enron.com',
  'susan.mara@enron.com',
  'jeff.dasovich@enron.com',
  'alan.comnes@enron.com',
  'paul.kaufman@enron.com',
  'joe.hartsoe@enron.com',
  'tim.belden@enron.com',
  'john.lavorato@enron.com',
  'lynn.blair@enron.com',
  'pat.shortridge@enron.com',
  'sue.nord@enron.com',
  'linda.robertson@enron.com',
  'janel.guerrero@enron.com',
  'louise.kitchen@enron.com',
  'terri.miller@enron.com',
  'jean.ryall@enron.com',
  'lora.sullivan@enron.com',
  'lindsay.meade@enron.com',
  'richard.ingersoll@enron.com',
  'elizabeth.linnell@enron.com',
  'janine.migden@enron.com',
  'bevin.hunter@enron.com',
  'nancy.hetrick@enron.com'],
 257643: ['rick.sierra@enron.com',
  'arthur.ransome@enron.com',
  'oscar.cedeno@enron.com',
  'dan.masters@enron.com',
  'mat